<a href="https://colab.research.google.com/github/maicon-reis/imersao-dados-desafio-final/blob/main/Imers%C3%A3o_Dados_Maicon_Reis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Imersão Dados - Alura**

A terceira edição da Imersão Dados da Alura é baseado em um desafio do Kaggle ([link do desafio](https://www.kaggle.com/c/lish-moa/overview/timeline)) que objetiva criar um algoritmo de classificação multi-label de compostos baseado em sua atividade biológica.

Nesse projeto serão utilizados dois datasets, sendo o primeiro é dividido em três grupos de dados:

* Três colunas com informações categóricas: Além do 'id' temos [1] **tratamento**, se a amostra foi submetida ao composto ou é uma amostra de controle; [2] **droga** que descreveve o tipo de tratamento que cada amostra foi submetida; [3] **tempo**, que cada amostra foi submetida ao tratamento e [4] **dose**, que indica a dosagem utilizada no tratamento;
* As **expressões gênicas** (genes) que são nomeadas como "g-" seguido de um número sequencial;
* As **linhagens celulares** (células) nomeadas como "c-" seguido de um número sequencial.

Tanto as expessões gênicas quanto as linhagens celulares contém resposta simultânea de diferentes linhagens celulares à utilização de determinado composto. Cada linha, contendo essas respostas é chamado de **assinatura celular**.

Já o segundo dataset é composto por mecanismos de ação do composto onde cada linha é resultado de um experimento na primeira base de dados, para tanto determinada coluna será marcada como 1. Nas colunas, além do **id**, são compostas por elementos que potencializam ou bloqueiam o receptor ou inibidores, utilizados para enzimas.





## **1. Análise do Dataset de Experimentos**

<img alt="Hipótese para o mecanismo de ação dos medicamentos homeopáticos.... |  Download Scientific Diagram" src="https://www.researchgate.net/publication/303321516/figure/fig2/AS:363187699503118@1463602103855/Hipotese-para-o-mecanismo-de-acao-dos-medicamentos-homeopaticos-Durante-o-processo-de.png"></img>

Na primeira parte do projeto vamos analisar o primeiro dataset contendo os experimentos.

Nas quatro primeiras colunas, além do id, temos o tratamento, o tempo, a dose e a droga (que mudaremos o nome para composto, para melhor adequar a terminologia). Após essas colunas temos uma sequência de colunas nomenclaturadas com g- e uma numeração e ademais c- e uma numeração sequencial.

Dessa forma cada linhagem celular, seja expressões gênicas ou de viabilidade celular, foi exposta a um determinado a um composto ou situando-se como amostra de controle, por um período de tempo, a uma determinada dosagem.

Os valores numéricos contidos nas colunas g_ correspondem ao resultado do caminho da informação biológica com destino à proteína, pela ativação ou não de um mecanismo de ação. Já o valor das colunas c_ é a viabilidade celular, ou seja, o quanto as células sobreviveram ou não à exposição do composto. O sucesso do que está se propondo.

In [ ]:
# bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

Desde já importaremos os dois datasets. Nessa primeira parte, só utilizaremos o arquivo **arquivo_experimentos**, na segunda parte utilizaremos ambos.

In [ ]:
# importação dos arquivos
arquivo_experimentos = "https://github.com/maicon-reis/imersao-dados-desafio-final/blob/main/Dados/dados_experimentos.zip?raw=true"
arquivo_resultados = "https://github.com/maicon-reis/imersao-dados-desafio-final/blob/main/Dados/dados_resultados.csv?raw=true"

Na leitura do arquivo para a criação do dataset (df_experimentos) foi utilizado o parâmetro nomeado `compression=` informando o tipo de compressão utilizada no arquivo. Dessa forma o próprio pandas descompacta o arquivo e faz o carregamento do dataset.

In [ ]:
# carregando o dataset com os experimentos
df_experimentos = pd.read_csv( arquivo_experimentos, compression='zip' )

# verificando as cinco primeiras linhas do dataset df_experimentos
df_experimentos.head()

In [ ]:
# verificando quantidade de linhas e colunas do datase
linhas, colunas = df_experimentos.shape
print(f'Número de linhas: {linhas}')
print(f'Número de colunas: {colunas}')

A partir da informação acima nós passamos a saber que o dataset tem 23814 linhas e 877 colunas. Das 877 colunas as cinco primeiras são compostas por id, tratamento, tempo, dose e droga. Precisamos saber agora quantas colunas são iniciados por g e por c.

In [ ]:
# separando colunas de genes e células
g_cols = [ g for g in df_experimentos.columns if g.startswith('g-')]
c_cols = [ c for c in df_experimentos.columns if c.startswith('c-')]

print(f'Número de g-: {len(g_cols)} (Indo de g-0 até g-{len(g_cols) - 1}).')
print(f'Número de c-: {len(c_cols)} (Indo de c-0 até c-{len(c_cols) - 1}).')

Verificamos que existem 772 colunas de gênicas e 100 colunas de viabilidades celulares. Iremos agora fazer algumas adequações no dataset para que possamos fazer uma visualização gráfica das informações para melhor entender os dados.

In [ ]:
# removendo os '-' dos nomes das colunas
df_experimentos.columns = [i.replace('-', '') for i in df_experimentos.columns]

# substituindo a palavra droga por composto para adequar a nomenclatura
mapa = {'droga': 'composto'}
df_experimentos.rename(columns=mapa, inplace=True)

In [ ]:
df_experimentos.info

No gráfico abaixo nós veremos os tipos e proporções referentes às séries tratamento, tempo e dose.

In [ ]:
# visualiza tipos e proporções de variáveis
plt.style.use('ggplot')
fig, axs = plt.subplots(1, 3, figsize=(15, 3))

axs[0].bar(df_experimentos['tratamento'].value_counts().index,
           df_experimentos['tratamento'].value_counts(normalize=True).values)
axs[0].set_ylabel('Quantidade')
axs[0].set_title('Tratamento')

axs[1].bar(['12','24','48'],
           df_experimentos['tempo'].value_counts(normalize=True).values)
axs[1].set_title('Tempo')

axs[2].bar(df_experimentos['dose'].value_counts().index,
           df_experimentos['dose'].value_counts(normalize=True).values)
axs[2].set_title('Dose')

plt.tight_layout()
plt.show()

Verifica-se que a variável tratamento apresenta dois tipos: com droga e com controle. Essa coluna é altamente desbalanceada, o que pode decorrer que a inclusão do tipo 'com_controle' par ser objeto de comparação, ou seja, as amostras controle foram submetidas às mesmas condições que as amostras tratadas com drogas, exceto pelo fato de não ter sido ministrado o composto, para com isso verificar o resultado do experimento nessas células.

Já a variável tempo é dividida em 12, 24 e 48 períodos de tempo. Por fim a variável dose é dividida em D1 e D2. Ambas as colunas tempo e dose, estão relativamente balanceadas.

No gráfico acima, nós conseguimos verificar características das colunas individualmente. Mas nós também queremos entender características dessas colunas em conjunto e formulei a seguinte pergunta: Como o tratamento foi distribuído por dose no tempo?

Chegamos no resultado abaixo.

In [ ]:
# Verificando as variáveis em conjunto de forma numérica
df_experimentos.groupby(['dose', 'tratamento', 'tempo']).size().unstack()

Como em programação o mesmo resultado pode ser obtido por vários meios, o resultado acima também poderia ser obtido utilizando a função crosstab, dessa forma:

```
# Utilizando crosstab para verificar colunas em conjunto
pd.crosstab([df_experimentos['dose'], df_experimentos['tratamento']], df_experimentos['tempo'])
```



In [ ]:
# Verificando as variáveis em conjunto de forma gráfica
ax = sns.catplot(x='tratamento', hue='dose', col='tempo',
                data=df_experimentos, kind='count', height=4)

plt.tight_layout()
plt.show()

A partir da tabela e gráficos acima, percebemos que há uma equivalência entre as doses aplicadas, tanto entre o tratamento com composto e com controle, bem como entre os períodos 24, 48 e 72.

A exceção é a aplicação da dose D1, com droga no período 48, que é bem maior do que a dose D2, com droga no mesmo período, e também é maior do que as doses D1 e D2 nos demais períodos. Entretanto essa diferença não é substancial a ponto de desbalancear esses tipos.

In [ ]:
# verificando quantidaded de elementos únicos da coluna composto
len(df_experimentos[ 'composto' ].unique())

Como a coluna composto contém 3286 tipos únicos, criaremos um gráfico contendo os 5 maiores tipos da coluna composto.

In [ ]:
# visualizando os cinco maiores tipos de compostos
cod_compostos = df_experimentos[ 'composto' ].value_counts()[ :5 ].index

plt.figure( figsize=( 8, 6 ) )
ax = sns.countplot( x='composto',
                   data=df_experimentos.query('composto in @cod_compostos'),
                   order = list(cod_compostos))
ax.set_title( 'Compostos Mais Utilizados', pad=20, fontsize=22 )
ax.set_ylabel( 'Número de Ocorrências' )
ax.set_xlabel( 'Composto' )

plt.tight_layout()
plt.show()

Após ter verificado as características das variáveis categóricas, vamos tentar entender algumas características das colunas que contém as expressões gênicas e as linhagens celulares. Dessa forma inicialmente criarei uma função que cria um gráfio de histograma para que possa ver a distribuição das colunas, notadamente as colunas de expressões gênicas e de linhagem celular. Então é necessário passar como parâmetro o nome da coluna que deseja ver a distribuição, caso a função seja chamada sem passar parâmetro, ela exibirá o gráfico da coluna 'g0'.

Como resultado, verifiquei que a distribuição das colunas de expressões gênicas se assemelham a uma distribuição normal, tendo a média, moda e mediana bastante próximas, muitas vezes se sobrepondo. Já em relação as linhagens celulares, algumas colunas apresentam em que o valor mais frequente é o -10, deslocando a moda para esse valor, e consequentemente deslocando a média para a esquerda da mediana. Tal característica, embora não seja objetivo desse projeto, deve ser melhor estudado para identificar o que a ocasiona.

In [ ]:
len(df_experimentos['g0'].unique())

In [ ]:
# cria função de histograma para verificar informações de distribuição das colunas 
def cria_histograma(coluna='g0'):
  media = df_experimentos[coluna].mean()
  moda = df_experimentos[coluna].mode()[0]
  mediana = df_experimentos[coluna].median()

  plt.figure( figsize=( 8,6 ) )
  df_experimentos[coluna].hist( bins=100, color='#bdbdbd' )

  plt.axvline(media, color='green', linestyle='--')
  plt.axvline(moda, color='blue', linestyle='--')
  plt.axvline(mediana, color='red', linestyle='--')

  plt.legend({'Média': media, 'Moda': moda, 'Mediana': mediana})
  plt.show()
  return None

In [ ]:
# chamando a função para criar histograma das colunas g e c
cria_histograma('c45')

Finalizando a fase de análise exploratória do dataset de experimentos eu criei uma função para visualizar a correlação entre as colunas, tanto de expressões gênicas e linhagens celulares através de um gráfico de heatmap.

Verifiquei, no geral, uma baixa correlação entre as colunas de expressões gênicas entre si, e uma alta correlação positiva entre as colunas de linhagens celulares.

A alta correlação dos tipos celulares pode ser que devido ao submissão do conjunto celular que os faz reagir de forma similar aos impactos do composto. Tais fatos devem ser motivo de aprofundamento de estudos, o que não é objetivo desse projeto.

In [ ]:
# cria gráfico de correlação entre as colunas
def cria_grafico_corr(inicio='g0', final='g50'):
  # Compute the correlation matrix
  corr = df_experimentos.loc[:, inicio:final].corr()

  # Generate a mask for the upper triangle
  mask = np.triu(np.ones_like(corr, dtype=bool))

  # Set up the matplotlib figure
  f, ax = plt.subplots(figsize=(11, 9))

  # Generate a custom diverging colormap
  cmap = sns.diverging_palette(230, 20, as_cmap=True)

  # Draw the heatmap with the mask and correct aspect ratio
  sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
              square=True, linewidths=.5, cbar_kws={"shrink": .5});
  return None

In [ ]:
# criando mapa de correlação entre as 50 primeiras colunas g
cria_grafico_corr(inicio='g0', final='g49')

In [ ]:
# criando mapa de correlação entre as 50 primeiras colunas c
cria_grafico_corr(inicio='c0', final='c49')

## **2. Análise do Dataset de Resultados**

O Dataset Resultados é composto pelos **Mecanismos de Ação** que resultam da assinatura celular. Além da coluna **"id"**, o dataset é composto por diversas nomenclaturas que representam os Mecanismos de Ação.

<img src='//upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Mechanism_of_action_for_beta_blockers.png/220px-Mechanism_of_action_for_beta_blockers.png' align='left'></img>
"Em farmacologia, a expressão mecanismo de ação refere-se à intereção bioquímica através da daqual uma droga produz um efeito farmacológico.Um mecanismo de ação usualmente inclui menção de um "alvo" molecular específico no qual a droga liga-se, tal como uma enzima ou receptor."

Site para consulta - [Link para ler mais](https://pt.wikipedia.org/wiki/Mecanismo_de_a%C3%A7%C3%A3o).

Verifica-se que as colunas são compostas por nomes terminados com "_inhibitor" (suprimem ou inibem a ação sobre uma enzima), bem como por "_agonist" e "_antagonist" (estes bloqueiam o receptor, enquanto aqueles ativam ou potencializam o receptor).

Cada linha da base de dados é o resultado de um experimento na base dedados. Quando o mecanismo de ação ele recebe o valor 1.

Através da análise descritiva realizada no dataset responderemos as seguintes perguntas:

<body>
  <ol>
    <li>Qual o mecanismo de ação que foi mais ativado?</li>
    <li>Os tipos de tratamento com_controle ativaram algum mecanismo de ação?</li>
    <li>Algum composto não ativou algum mecanismo de ação?</li>
  </ol>
</body>

In [ ]:
# carregando o dataset resultados
df_resultados = pd.read_csv(arquivo_resultados)

# visualização das cinco primeiras linhas do dataset resultados
df_resultados.head()

In [ ]:
# verificando quantidade de linhas e colunas do datase
linhas, colunas = df_resultados.shape
print(f'Número de linhas: {linhas}')
print(f'Número de colunas: {colunas}')

Através da consulta acima, verificamos que o dataset resultados contém 207 colunas, excetuando-se a coluna "id", conclui-se, então que o dataset possui 206 mecanismos de ação.

Tais mecanismos de ação recebem o valor 1, a partir  da assinatura celular, quando eles foram ativados, e o valor 0, quando não foram ativados. Como pode-se ver abaixo como o exemplo do 'acat_inhibitor'.

In [ ]:
df_resultados['acat_inhibitor'].unique()

In [ ]:
# verificando Mecanismos de Ação de colunas por atuação
inhibitor = []
agonist = []
antagonist = []
for coluna in df_resultados.columns:
  if coluna.endswith('_inhibitor'):
    inhibitor.append(coluna)
  elif coluna.endswith('_agonist'):
    agonist.append(coluna)
  else:
    antagonist.append(coluna)

print(f'Inhibitor: {len(inhibitor)}')
print(f'Agonist: {len(agonist)}')
print(f'Antagonist: {len(antagonist)}')

Através do código acima, pude concluir que das 207 colunas com mecanismos de ação 112 são inhibitor, 28 agonist e 67 antagonist.

In [ ]:
# mecanismo de ação mais ativado
contagem_moa = df_resultados.select_dtypes( 'int64' ).sum().sort_values( ascending=False )[:10]
contagem_moa

Através do fragmento de código acima, verfica-se que o mecanismo de ação 'nfkb_inibitor' foi ativado 832 vezes, sendo então o mecanismo de ação mais ativado.

Também foi visualizado que dentre os 10 mecanismos de ação mais ativados, 6 são do tipo 'inhibitor' e 4 do tipo 'antagonist'.

Dando prosseguimento vamos criar duas novas colunas no dataset resultados, informando quand

In [ ]:
# criando coluna que conta quantos mecanismos de ação foram ativados por assinatura celular
df_resultados.loc[:, 'n_moa'] = df_resultados.drop('id', axis=1).sum( axis=1 )

# criando coluna de booleano se o mecanismo de ação foi ativado ou não
df_resultados.loc[:, 'ativo_moa'] = (df_resultados['n_moa'] != 0)

# verificando as cinco primeiros linhas do dataset resultados
df_resultados.head()

Verifiquei que alguns mecanismos de ação não foram ativados. Presume-se que o tipo de tratamento com controle não deve acionar nenhum mecanismo de ação. Dessa forma, fiz um junção entre os dois datasets experimentos e resultados, deste dataset apenas levando as colunas recém criadas "n_moa" e "ativo_moa" que condensam a análise.

A partir do novo DataFrame criado verifiquei se o tipo de tratamento "com_controle" ativou algum mecanismo de ação.

In [ ]:
# fazendo a jução dos DataFrames df_experimentos e df_resultados
df_combinados = pd.merge(df_experimentos, df_resultados[['id', 'n_moa', 'ativo_moa']], on='id')

# verificando as cinco primeiras linhas do DataFrame criado
df_combinados.head()

In [ ]:
# verificando se o tratamento com controle não ativou nenhum mecanismo de ação.
df_combinados.query('tratamento == "com_controle"')['ativo_moa'].unique()

In [ ]:
# verificando se o tratamento com controle não ativou nenhum mecanismo de ação.
df_combinados.query('tratamento == "com_droga"')['ativo_moa'].value_counts()

O resultado, como experado, foi que o tipo de tratamento "com_controle" não ativou nenhum mecanismo de ação. Por outro lado, verificamos que 7501 tipos de tratamento "com_droga" também não ativaram nenhum mecanismo de ação. Fato que deve ser motivo de estudos com objetivo de identificar o motivo.

## **3. Avaliação Preditiva**

Nesta parte do projeto, meu objetivo concentrou-se em saber se o conjunto de dados seria capaz de, dado um conjunto de atributos, é possível saber se a dose aplicada àquele conjunto é D1 ou D2 através da análise do conjunto de dados.

Para tanto irei fazer algumas transformações no DataFrame. O atributo alvo será convertido de variável categórica em boleano, em que o tipo de dose D1 será transformado no booleano 0, e o tipo de dose D2 no booleano 1.

Ainda na etapa de pré-processamento os atributos categórios serão codificados em numéricos através da utilização de uma função do pandas chamada ```get_dummies()```. O mesmo resultado poderia ser obtido através da utilização do módulo ```OneHotEncoder()``` da biblioteca Sklearn.

Após a fase de pré-processamento dos dados, estes serão divididos em **atributos** (variáveis selecionadas) e a variável **alvo**. Estes por sua vez serão subdivididos em dados do treino e dados de teste. Ituitivamente, aqueles servirão para treinar o modelo e estes para testá-lo e avaliá-lo.

O modelo utilizado para essa avaliação é o algoritmo de classificação por apredizado supervisionado **Random Forest** também pertencente à biblioteca de machine learning Sklearn, podendo ser importada a partir da sintaxe:

```from sklearn.ensemble import RandomForestClassifier```

In [ ]:
# convertendo a coluna algo "dose" para numérico
enc = LabelEncoder()
dose_int = enc.fit_transform( df_combinados['dose'] )
df_combinados['dose_int'] = dose_int
df_combinados.drop( 'dose', axis=1, inplace=True )

In [ ]:
# realizando a separação dos dados e transformando colunas categóricas em numéricas
X = df_combinados.drop( [ 'id', 'composto', 'ativo_moa', 'dose_int' ], axis=1 )
X = pd.get_dummies( X, columns=[ 'tratamento', 'tempo' ] )
y = df_combinados['dose_int']

# dividindo os dados em treino e teste
X_treino, X_teste, y_treino, y_teste =  train_test_split( X, y,
                                                         test_size=0.2,
                                                         random_state=0)

# instanciando o classificador
rf_clf = RandomForestClassifier(criterion='entropy',
                                max_depth=10,
                                min_samples_leaf=5,
                                min_samples_split=2,
                                n_estimators=150, random_state=0)
# treinando o modelo
rf_clf.fit( X_treino, y_treino )

# predizendo o modelo com os dados de teste
resultado = rf_clf.predict( X_teste )

# avaliando o modelo com base na acurácia
acuracia = accuracy_score( y_teste, resultado )

print(f'Acurácia: {acuracia}')

Do experimento aplicado foi obtido uma taxa de acurácia de 0.957589, o que é um valor significativo de assertividade. Cabe salientar que esse modelo pode ser afinado (*tunning*) com a utilização do módulo ```GridSeachCV()``` da biblioteca Sklearn.

Por fim segue um relatório de classificação contendo outra métricas de avaliação do modelo.

In [ ]:
# criando uma tabela com algumas métricas de avaliação do modelo
print(
    classification_report( y_teste, resultado )
)